## Upload Pcap

### Imports & Setup

In [ ]:
import os
import sys
import steelscript
from steelscript.netshark.core import NetShark
from steelscript.netshark.core.types import Value, Key
from steelscript.common import UserAuth

In [ ]:
hostname = "NETSHARK.HOSTNAME.COM"
username = "USERNAME"
password = "PASSWORD"

In [ ]:
netshark = NetShark(hostname, auth=UserAuth(username, password))
netshark

### Upload trace file

(Note, this may yield an exception...)

In [ ]:
pkgs = steelscript.__path__
if len(pkgs) == 1:
    pcap_path = os.path.join(sys.prefix, 'share', 'doc', 'steelscript', 'docs', 'netshark', 'tutorial.pcap')
else:
    ns_path = [x for x in pkgs if 'steelscript-netshark' in x][0]
    pcap_path = os.path.join(ns_path, '..', 'docs', 'tutorial.pcap')

if not os.path.exists(pcap_path):
    print 'ERROR: Unable to find sample pcap file in usual places!'

In [ ]:
localfile = pcap_path
remotefile = '/%s/tutorial.pcap' % username
tracefile = netshark.upload_trace_file(remotefile, localfile)

### Check, then upload trace file
Better to check first if the remote file already exists. 
If it does, remove it first just in case the remote file 
does not match what we want

In [ ]:
if netshark.exists(remotefile):
    print "Removing existing file"
    tracefile = netshark.get_file(remotefile)
    tracefile.remove()
    
tracefile = netshark.upload_trace_file(remotefile, localfile)
tracefile

In [ ]:
tracefile.get_info()

### Check out the file system a bit

In [ ]:
homedir = netshark.get_dir('/%s' % username)
homedir

In [ ]:
homedir.list()

In [ ]:
homedir.get_info()

### Create a simple view based on the tracefile

Note that a timefilter is not used below -- the view will then run over the entire pcap file.

In [ ]:
columns = [
    Key(netshark.columns.tcp.server_ip),
    Key(netshark.columns.tcp.server_port),
    Key(netshark.columns.generic.traffic_type.application),
    Value(netshark.columns.generic.bytes)
]

In [ ]:
view = netshark.create_view(tracefile, columns)

In [ ]:
data = view.get_data(aggregated=True)

In [ ]:
data[0]